# Upward recursion relations for $\mathcal{I}_v$ and $\mathcal{J}_v$

#### Validation of the analytical solution to the integrals

Let's import some stuff:

In [24]:
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as pl
from mpmath import ellipe, ellipk
from sympy import factorial
%matplotlib inline
epsabs=1e-12
epsrel=1e-12

Let's define the numerical form of the integrals:

In [2]:
def I(v, k):
    """Return the integral I, evaluated numerically."""
    kappa = 2 * np.arcsin(k)
    func = lambda x: np.sin(x) ** (2 * v)
    res, err = quad(func, -0.5 * kappa, 0.5 * kappa, epsabs=epsabs, epsrel=epsrel)
    return res

def J(v, k):
    """Return the integral J, evaluated numerically."""
    kappa = 2 * np.arcsin(k)
    func = lambda x: np.sin(x) ** (2 * v) * (1 - k ** (-2) * np.sin(x) ** 2) ** 1.5
    res, err = quad(func, -0.5 * kappa, 0.5 * kappa, epsabs=epsabs, epsrel=epsrel)
    return res

Let's also define our analytical recursion relations, as well as the initial values:

In [32]:
def IUpward(v, k, nterms=30):
    """Return the integral I, evaluated recursively."""
    if v == 0:
        return 2 * np.arcsin(k)
    else:
        return (1 / v) * ((2 * v - 1) / 2 * IUpward(v - 1, k, nterms=nterms) - k ** (2 * v - 1) * np.sqrt(1 - k ** 2))

def JUpward(v, k, nterms=30):
    """Return the integral J, evaluated recursively."""
    if v == 0:
        return 2 / (3 * k ** 3) * (2 * (2 * k ** 2 - 1) * ellipe(k ** 2) + 
                                   (1 - k ** 2) * (2 - 3 * k ** 2) * ellipk(k ** 2))
    elif v == 1:
        return 2 / (15 * k ** 3) * ((-3 * k ** 4 + 13 * k ** 2 - 8) * ellipe(k ** 2) + 
                                   (1 - k ** 2) * (8 - 9 * k ** 2) * ellipk(k ** 2))
    else:
        return (1 / (2 * v + 3)) * (2 * (v + (v - 1) * k ** 2 + 1) * JUpward(v - 1, k, nterms=nterms) -
                                    k ** 2 * (2 * v - 3) * JUpward(v - 2, k, nterms=nterms))

Let's show that the error is close to the machine limit for $\frac{1}{2} < k < 1$ and $v < 10$. (Recall from the text that these expressions are numerically unstable for $k < \frac{1}{2}$, in which case we perform downward recursion instead.)

In [53]:
maxrel = 0
maxfrac = 0
for k in np.linspace(0.5, 1.0, 100):
    for v in range(10):
        I1 = I(v, k)
        I2 = IUpward(v, k)
        rel = np.abs(I1 - I2)
        frac = np.abs(rel / I1)
        if rel > maxrel:
            maxrel = rel
        if frac > maxfrac:
            maxfrac = frac
print("The maximum relative error on I is %.4e." % maxrel)
print("The maximum fractional error on I is %.4e." % maxfrac)

The maximum relative error on I is 5.5511e-16.
The maximum fractional error on I is 1.3513e-10.


In [51]:
maxrel = 0
maxfrac = 0
for k in np.linspace(0.5, 1.0, 100):
    for v in range(10):
        J1 = J(v, k)
        J2 = JUpward(v, k)
        rel = np.abs(J1 - J2)
        frac = np.abs(rel / J1)
        if rel > maxrel:
            maxrel = rel
        if frac > maxfrac:
            maxfrac = frac
print("The maximum relative error on J is %.4e." % maxrel)
print("The maximum fractional error on J is %.4e." % maxfrac)

The maximum relative error on J is 3.9684e-14.
The maximum fractional error on J is 7.9346e-08.


■